In [86]:
import pandas as pd
import re
import numpy as np
import textwrap

import matplotlib.pyplot as plt
import seaborn as sns

In [87]:
data = pd.read_csv('../fraud_tools_team_ds_test/synthetic_dirty_transaction_logs.csv')

In [88]:
data.head()

,raw_log
0,2025-07-05 19:18:10::user1069::withdrawal::299...
1,NaN
2,MALFORMED_LOG
3,usr:user1076|cashout|€4821.85|Glasgow|2025-07-...
4,2025-07-20 05:38:14 >> [user1034] did top-up -...


In [89]:
data[data['raw_log'] == None]

,raw_log


In [90]:
data[data['raw_log'] == 'MALFORMED_LOG']

,raw_log
2,MALFORMED_LOG
7,MALFORMED_LOG
12,MALFORMED_LOG
16,MALFORMED_LOG
17,MALFORMED_LOG
...,...
9932,MALFORMED_LOG
9934,MALFORMED_LOG
9944,MALFORMED_LOG
9964,MALFORMED_LOG


In [91]:
data[data.isnull().any(axis=1)]

,raw_log
1,NaN
28,NaN
45,NaN
57,NaN
61,NaN
...,...
9971,NaN
9986,NaN
9990,NaN
9996,NaN


In [92]:
data['raw_log'] = data['raw_log'].apply(lambda x: np.nan if x == 'MALFORMED_LOG' else x)

In [93]:
data.isnull().sum()

raw_log    2226
dtype: int64

In [94]:
data = data.dropna(subset=['raw_log'], ignore_index=True)

In [95]:
data.shape

(7774, 1)

In [96]:
data

,raw_log
0,2025-07-05 19:18:10::user1069::withdrawal::299...
1,usr:user1076|cashout|€4821.85|Glasgow|2025-07-...
2,2025-07-20 05:38:14 >> [user1034] did top-up -...
3,2025-06-13 10:04:51 >> [user1068] did deposit ...
4,2025-07-29 23:47:37 | user: user1014 | txn: de...
...,...
7769,usr:user1058|debit|£3992.06|Leeds|2025-06-10 0...
7770,08/07/2025 08:50:09 ::: user1069 *** WITHDRAWA...
7771,2025-07-31 07:41:34::user1024::deposit::1331.2...
7772,2025-06-23 05:14:03 >> [user1029] did withdraw...


In [97]:
data.isnull().sum()

raw_log    0
dtype: int64

In [98]:
data['raw_log'].head() # .to_list()

0    2025-07-05 19:18:10::user1069::withdrawal::299...
1    usr:user1076|cashout|€4821.85|Glasgow|2025-07-...
2    2025-07-20 05:38:14 >> [user1034] did top-up -...
3    2025-06-13 10:04:51 >> [user1068] did deposit ...
4    2025-07-29 23:47:37 | user: user1014 | txn: de...
Name: raw_log, dtype: object

# Potential Features:
- User: user1043, user1034, etc.
- Timestamp: 2025-07-05 06:12:48, 04/07/2025 01:08:12
- Action: withdrawal, deposit, purchase, refund, etc.
- Amount: £4803.93, $1194.19, 2804.97£
- Currency: £, $, €
- Location: Liverpool, Glasgow, Birmingham
- Device: IiPhone 13, Pixel 6, Huawei P30, etc.

# Extract User_id

In [99]:
data.head()

,raw_log
0,2025-07-05 19:18:10::user1069::withdrawal::299...
1,usr:user1076|cashout|€4821.85|Glasgow|2025-07-...
2,2025-07-20 05:38:14 >> [user1034] did top-up -...
3,2025-06-13 10:04:51 >> [user1068] did deposit ...
4,2025-07-29 23:47:37 | user: user1014 | txn: de...


In [100]:
data['raw_log'] #.to_list()

0       2025-07-05 19:18:10::user1069::withdrawal::299...
1       usr:user1076|cashout|€4821.85|Glasgow|2025-07-...
2       2025-07-20 05:38:14 >> [user1034] did top-up -...
3       2025-06-13 10:04:51 >> [user1068] did deposit ...
4       2025-07-29 23:47:37 | user: user1014 | txn: de...
                              ...                        
7769    usr:user1058|debit|£3992.06|Leeds|2025-06-10 0...
7770    08/07/2025 08:50:09 ::: user1069 *** WITHDRAWA...
7771    2025-07-31 07:41:34::user1024::deposit::1331.2...
7772    2025-06-23 05:14:03 >> [user1029] did withdraw...
7773    2025-07-27 17:57:22::user1048::refund::2988.49...
Name: raw_log, Length: 7774, dtype: object

In [101]:
user_id_pattern = re.compile(r"""
   .*?(?:user[:=]?|user\s)?(?P<user>user\d{4}) 
""", re.VERBOSE)

text = textwrap.dedent("""
                '2025-07-05 19:18:10::user1069::withdrawal::2995.12::London::iPhone 13',
                'usr:user1076|cashout|€4821.85|Glasgow|2025-07-15 12:56:05|Pixel 6',
                '2025-07-20 05:38:14 >> [user1034] did top-up - amt=€2191.06 - None // dev:iPhone 13',
                '2025-06-13 10:04:51 >> [user1068] did deposit - amt=€1691.09 - Glasgow // dev:None',
                '2025-07-29 23:47:37 | user: user1014 | txn: deposit of £3539.5 from Glasgow | device: iPhone 13',
                '2025-06-23 14:45:58 - user=user1075 - action=debit $1215.74 - ATM: Leeds - device=Samsung Galaxy S10',
                '2025-07-31 06:50:50 | user: user1071 | txn: cashout of $1772.13 from None | device: Nokia 3310',
                '2025-07-07 20:42:12 - user=user1098 - action=deposit €304.0 - ATM: Birmingham - device=Nokia 3310',
                """)

matches = user_id_pattern.findall(text)
print("Matches found:", matches)


Matches found: ['user1069', 'user1076', 'user1034', 'user1068', 'user1014', 'user1075', 'user1071', 'user1098']


In [102]:
data['raw_log'].apply(lambda x: user_id_pattern.findall(x)[0])

0       user1069
1       user1076
2       user1034
3       user1068
4       user1014
          ...   
7769    user1058
7770    user1069
7771    user1024
7772    user1029
7773    user1048
Name: raw_log, Length: 7774, dtype: object

In [103]:
# Check for missing patterns

data[data['raw_log'].apply(lambda x: user_id_pattern.findall(x)[0]) == np.nan]

,raw_log


In [104]:
data['user_id'] = data['raw_log'].apply(lambda x: user_id_pattern.findall(x)[0])

In [105]:
data.isnull().sum()

raw_log    0
user_id    0
dtype: int64

# Extract Time-Stamp

In [106]:
time_stamp_pattern = re.compile(r"""
    (?P<timestamp>                                              # Timestamps
        \d{4}-\d{2}-\d{2} \s \d{2}:\d{2}:\d{2}(?=::)?            # e.g., 2025-07-05 19:18:10
        |
        \d{2}/\d{2}/\d{4} \s \d{2}:\d{2}:\d{2}                   # e.g., 04/07/2025 01:08:12
    )
""", re.VERBOSE)

text = textwrap.dedent("""
        '2025-07-05 19:18:10::user1069::withdrawal::2995.12::London::iPhone 13',
        'usr:user1076|cashout|€4821.85|Glasgow|2025-07-15 12:56:05|Pixel 6',
        '2025-07-20 05:38:14 >> [user1034] did top-up - amt=€2191.06 - None // dev:iPhone 13',
        '2025-06-13 10:04:51 >> [user1068] did deposit - amt=€1691.09 - Glasgow // dev:None',
        '2025-06-23 14:45:58 - user=user1075 - action=debit $1215.74 - ATM: Leeds - device=Samsung Galaxy S10',
        '2025-07-31 06:50:50 | user: user1071 | txn: cashout of $1772.13 from None | device: Nokia 3310',
        '24/07/2025 22:47:06 ::: user1080 *** PURCHASE ::: amt:951.85$ @ Liverpool <Xiaomi Mi 11>',
        '2025-06-10 20:06:30 >> [user1025] did withdrawal - amt=€3261.07 - London // dev:iPhone 13',
        '14/06/2025 07:25:39 ::: user1019 *** CASHOUT ::: amt:2428.72£ @ Leeds <iPhone 13>',
        'user1093 2025-07-05 14:11:06 withdrawal 4926.56 None Huawei P30',
        '2025-07-05 06:12:48 - user=user1043 - action=withdrawal £4803.93 - ATM: Liverpool - device=iPhone 13',
        '2025-07-23 15:57:12 | user: user1098 | txn: purchase of €2019.47 from Glasgow | device: None',
        '2025-06-23 10:05:51 >> [user1048] did deposit - amt=€2096.71 - Birmingham // dev:Huawei P30',
        'usr:user1034|top-up|$1194.19|Liverpool|2025-06-24 05:06:27|Pixel 6',
        '2025-06-25 07:18:59 - user=user1008 - action=top-up £4645.16 - ATM: London - device=Xiaomi Mi 11',
        '2025-07-17 23:05:52 >> [user1011] did purchase - amt=£2529.92 - Birmingham // dev:iPhone 13',
        '2025-06-22 21:50:40 - user=user1003 - action=refund £4658.85 - ATM: Liverpool - device=Huawei P30',
        '2025-07-04 02:49:11::user1083::transfer::1362.47::Glasgow::Pixel 6',
        'user1077 2025-06-26 03:04:52 refund 2527.54 London iPhone 13',
        '04/07/2025 01:08:12 ::: user1046 *** CASHOUT ::: amt:2804.97£ @ Manchester <Pixel 6>',
        '09/06/2025 22:58:25 ::: user1074 *** WITHDRAWAL ::: amt:4173.14€ @ Liverpool <Xiaomi Mi 11>',
        '2025-07-08 14:07:36 >> [user1086] did refund - amt=£1823.89 - Birmingham // dev:Samsung Galaxy S10',
""")

matches = time_stamp_pattern.findall(text)
print("Matches found:", matches)


Matches found: ['2025-07-05 19:18:10', '2025-07-15 12:56:05', '2025-07-20 05:38:14', '2025-06-13 10:04:51', '2025-06-23 14:45:58', '2025-07-31 06:50:50', '24/07/2025 22:47:06', '2025-06-10 20:06:30', '14/06/2025 07:25:39', '2025-07-05 14:11:06', '2025-07-05 06:12:48', '2025-07-23 15:57:12', '2025-06-23 10:05:51', '2025-06-24 05:06:27', '2025-06-25 07:18:59', '2025-07-17 23:05:52', '2025-06-22 21:50:40', '2025-07-04 02:49:11', '2025-06-26 03:04:52', '04/07/2025 01:08:12', '09/06/2025 22:58:25', '2025-07-08 14:07:36']


In [107]:
data['raw_log'].apply(lambda x: time_stamp_pattern.findall(x)[0])

0       2025-07-05 19:18:10
1       2025-07-15 12:56:05
2       2025-07-20 05:38:14
3       2025-06-13 10:04:51
4       2025-07-29 23:47:37
               ...         
7769    2025-06-10 03:55:41
7770    08/07/2025 08:50:09
7771    2025-07-31 07:41:34
7772    2025-06-23 05:14:03
7773    2025-07-27 17:57:22
Name: raw_log, Length: 7774, dtype: object

In [108]:
data['timestamp'] = data['raw_log'].apply(lambda x: time_stamp_pattern.findall(x)[0])

In [109]:
data.isnull().sum()

raw_log      0
user_id      0
timestamp    0
dtype: int64

# Extract Action: withdrawal, deposit, purchase, refund, etc.

In [110]:


action_pattern = re.compile(r"""
       (?:did|action=|\*\*\*|::)?\s*                # non-capturing: possible preamble
    (?P<action>deposit|withdrawal|cashout|top-?up|purchase|debit|refund|transfer|purchase)  # the action     # deposite, withdrawal, refund
    
""", re.VERBOSE)

text = textwrap.dedent("""
        '2025-07-05 19:18:10::user1069::withdrawal::2995.12::London::iPhone 13',
        'usr:user1076|cashout|€4821.85|Glasgow|2025-07-15 12:56:05|Pixel 6',
        '2025-07-20 05:38:14 >> [user1034] did top-up - amt=€2191.06 - None // dev:iPhone 13',
        '2025-06-13 10:04:51 >> [user1068] did deposit - amt=€1691.09 - Glasgow // dev:None',
        '2025-06-23 14:45:58 - user=user1075 - action=debit $1215.74 - ATM: Leeds - device=Samsung Galaxy S10',
        '2025-07-31 06:50:50 | user: user1071 | txn: cashout of $1772.13 from None | device: Nokia 3310',
        '24/07/2025 22:47:06 ::: user1080 *** PURCHASE ::: amt:951.85$ @ Liverpool <Xiaomi Mi 11>'
""")

matches = action_pattern.findall(text.lower())
print("Matches found:", len(matches), matches)


Matches found: 7 ['withdrawal', 'cashout', 'top-up', 'deposit', 'debit', 'cashout', 'purchase']


In [111]:
action_pattern = re.compile(r"""
       (?:did|action=|\*\*\*|::)?\s*                # non-capturing: possible preamble
    (?P<action>deposit|withdrawal|cashout|top-?up|purchase|debit|refund|transfer|purchase)  # the action     # deposite, withdrawal, refund
    
""", re.VERBOSE)

text = textwrap.dedent("""
        '2025-07-05 19:18:10::user1069::withdrawal::2995.12::London::iPhone 13',
        'usr:user1076|cashout|€4821.85|Glasgow|2025-07-15 12:56:05|Pixel 6',
        '2025-07-20 05:38:14 >> [user1034] did top-up - amt=€2191.06 - None // dev:iPhone 13',
        '2025-06-13 10:04:51 >> [user1068] did deposit - amt=€1691.09 - Glasgow // dev:None',
        '2025-06-23 14:45:58 - user=user1075 - action=debit $1215.74 - ATM: Leeds - device=Samsung Galaxy S10',
        '2025-07-31 06:50:50 | user: user1071 | txn: cashout of $1772.13 from None | device: Nokia 3310',
        '24/07/2025 22:47:06 ::: user1080 *** PURCHASE ::: amt:951.85$ @ Liverpool <Xiaomi Mi 11>',
        '2025-06-10 20:06:30 >> [user1025] did withdrawal - amt=€3261.07 - London // dev:iPhone 13',
        '14/06/2025 07:25:39 ::: user1019 *** CASHOUT ::: amt:2428.72£ @ Leeds <iPhone 13>',
        'user1093 2025-07-05 14:11:06 withdrawal 4926.56 None Huawei P30',
        '2025-07-05 06:12:48 - user=user1043 - action=withdrawal £4803.93 - ATM: Liverpool - device=iPhone 13',
        '2025-07-23 15:57:12 | user: user1098 | txn: purchase of €2019.47 from Glasgow | device: None',
        '2025-06-23 10:05:51 >> [user1048] did deposit - amt=€2096.71 - Birmingham // dev:Huawei P30',
        'usr:user1034|top-up|$1194.19|Liverpool|2025-06-24 05:06:27|Pixel 6',
        '2025-06-25 07:18:59 - user=user1008 - action=top-up £4645.16 - ATM: London - device=Xiaomi Mi 11',
        '2025-07-17 23:05:52 >> [user1011] did purchase - amt=£2529.92 - Birmingham // dev:iPhone 13',
        '2025-06-22 21:50:40 - user=user1003 - action=refund £4658.85 - ATM: Liverpool - device=Huawei P30',
        '2025-07-04 02:49:11::user1083::transfer::1362.47::Glasgow::Pixel 6',
        'user1077 2025-06-26 03:04:52 refund 2527.54 London iPhone 13',
        '04/07/2025 01:08:12 ::: user1046 *** CASHOUT ::: amt:2804.97£ @ Manchester <Pixel 6>',
        '09/06/2025 22:58:25 ::: user1074 *** WITHDRAWAL ::: amt:4173.14€ @ Liverpool <Xiaomi Mi 11>',
        '2025-07-08 14:07:36 >> [user1086] did refund - amt=£1823.89 - Birmingham // dev:Samsung Galaxy S10',
""")

matches = action_pattern.findall(text.lower())
print("Matches found:", len(matches), matches)


Matches found: 22 ['withdrawal', 'cashout', 'top-up', 'deposit', 'debit', 'cashout', 'purchase', 'withdrawal', 'cashout', 'withdrawal', 'withdrawal', 'purchase', 'deposit', 'top-up', 'top-up', 'purchase', 'refund', 'transfer', 'refund', 'cashout', 'withdrawal', 'refund']


In [112]:
data['raw_log'].apply(lambda x: (action_pattern.findall(x.lower()))[0])

0       withdrawal
1          cashout
2           top-up
3          deposit
4          deposit
           ...    
7769         debit
7770    withdrawal
7771       deposit
7772    withdrawal
7773        refund
Name: raw_log, Length: 7774, dtype: object

In [113]:
# Check for missing patterns
data[data['raw_log'].apply(lambda x: (action_pattern.findall(x.lower()))[0]) == np.nan]

,raw_log,user_id,timestamp


In [114]:
data['action'] = data['raw_log'].apply(lambda x: (action_pattern.findall(x.lower()))[0])

In [115]:
data.isnull().sum()

raw_log      0
user_id      0
timestamp    0
action       0
dtype: int64

# Extract Amount

In [116]:
amount_pattern = re.compile(r"""
    (?:amt[:=]?\s*|of\s+)?                # optional prefix
    (?:[€$£])?                            # optional non-captured currency
    (?P<amount>                           # named group (amount only)
        (?:\d{1,3}(?:,\d{3})*|\d+)        # integer part
        \.\d{1,2}                           # decimal part
    )
""", re.VERBOSE)

text = textwrap.dedent("""
                '2025-07-05 19:18:10::user1069::withdrawal::2995.12::London::iPhone 13',
                'usr:user1076|cashout|€4821.85|Glasgow|2025-07-15 12:56:05|Pixel 6',
                '2025-07-20 05:38:14 >> [user1034] did top-up - amt=€2191.06 - None // dev:iPhone 13',
                '2025-06-13 10:04:51 >> [user1068] did deposit - amt=€1691.09 - Glasgow // dev:None',
                '2025-07-29 23:47:37 | user: user1014 | txn: deposit of £3539.5 from Glasgow | device: iPhone 13',
                '2025-06-23 14:45:58 - user=user1075 - action=debit $1215.74 - ATM: Leeds - device=Samsung Galaxy S10',
                '2025-07-31 06:50:50 | user: user1071 | txn: cashout of $1772.13 from None | device: Nokia 3310',
                '2025-07-07 20:42:12 - user=user1098 - action=deposit €304.0 - ATM: Birmingham - device=Nokia 3310',
                '24/07/2025 22:47:06 ::: user1080 *** PURCHASE ::: amt:951.85$ @ Liverpool <Xiaomi Mi 11>',
                '2025-06-10 20:06:30 >> [user1025] did withdrawal - amt=€3261.07 - London // dev:iPhone 13',
                '14/06/2025 07:25:39 ::: user1019 *** CASHOUT ::: amt:2428.72£ @ Leeds <iPhone 13>',
                'user1093 2025-07-05 14:11:06 withdrawal 4926.56 None Huawei P30',
                '2025-07-05 06:12:48 - user=user1043 - action=withdrawal £4803.93 - ATM: Liverpool - device=iPhone 13',
                '2025-07-23 15:57:12 | user: user1098 | txn: purchase of €2019.47 from Glasgow | device: None',
                '2025-06-23 10:05:51 >> [user1048] did deposit - amt=€2096.71 - Birmingham // dev:Huawei P30',
                'usr:user1034|top-up|$1194.19|Liverpool|2025-06-24 05:06:27|Pixel 6',
                '2025-06-25 07:18:59 - user=user1008 - action=top-up £4645.16 - ATM: London - device=Xiaomi Mi 11',
                '2025-07-17 23:05:52 >> [user1011] did purchase - amt=£2529.92 - Birmingham // dev:iPhone 13',
                '2025-06-22 21:50:40 - user=user1003 - action=refund £4658.85 - ATM: Liverpool - device=Huawei P30',
                '2025-07-04 02:49:11::user1083::transfer::1362.47::Glasgow::Pixel 6',
                'user1077 2025-06-26 03:04:52 refund 2527.54 London iPhone 13',
                '04/07/2025 01:08:12 ::: user1046 *** CASHOUT ::: amt:2804.97£ @ Manchester <Pixel 6>',
                '09/06/2025 22:58:25 ::: user1074 *** WITHDRAWAL ::: amt:4173.14€ @ Liverpool <Xiaomi Mi 11>',
                '2025-07-08 14:07:36 >> [user1086] did refund - amt=£1823.89 - Birmingham // dev:Samsung Galaxy S10',
""")

matches = amount_pattern.findall(text)
print("Matches found:", len(matches), matches)

Matches found: 24 ['2995.12', '4821.85', '2191.06', '1691.09', '3539.5', '1215.74', '1772.13', '304.0', '951.85', '3261.07', '2428.72', '4926.56', '4803.93', '2019.47', '2096.71', '1194.19', '4645.16', '2529.92', '4658.85', '1362.47', '2527.54', '2804.97', '4173.14', '1823.89']


In [117]:
data['raw_log'].head(6).to_list()

['2025-07-05 19:18:10::user1069::withdrawal::2995.12::London::iPhone 13',
 'usr:user1076|cashout|€4821.85|Glasgow|2025-07-15 12:56:05|Pixel 6',
 '2025-07-20 05:38:14 >> [user1034] did top-up - amt=€2191.06 - None // dev:iPhone 13',
 '2025-06-13 10:04:51 >> [user1068] did deposit - amt=€1691.09 - Glasgow // dev:None',
 '2025-07-29 23:47:37 | user: user1014 | txn: deposit of £3539.5 from Glasgow | device: iPhone 13',
 '2025-06-23 14:45:58 - user=user1075 - action=debit $1215.74 - ATM: Leeds - device=Samsung Galaxy S10']

In [118]:
data['raw_log'].apply(lambda x: (amount_pattern.findall(x))[0])

0       2995.12
1       4821.85
2       2191.06
3       1691.09
4        3539.5
         ...   
7769    3992.06
7770    1401.58
7771     1331.2
7772    4203.63
7773    2988.49
Name: raw_log, Length: 7774, dtype: object

In [119]:
# Check for missing patterns
data[data['raw_log'].apply(lambda x: (amount_pattern.findall(x.lower()))[0]) == np.nan]

,raw_log,user_id,timestamp,action


In [120]:
data['amount'] = data['raw_log'].apply(lambda x: (amount_pattern.findall(x.lower()))[0])

In [121]:
data.head()

,raw_log,user_id,timestamp,action,amount
0,2025-07-05 19:18:10::user1069::withdrawal::299...,user1069,2025-07-05 19:18:10,withdrawal,2995.12
1,usr:user1076|cashout|€4821.85|Glasgow|2025-07-...,user1076,2025-07-15 12:56:05,cashout,4821.85
2,2025-07-20 05:38:14 >> [user1034] did top-up -...,user1034,2025-07-20 05:38:14,top-up,2191.06
3,2025-06-13 10:04:51 >> [user1068] did deposit ...,user1068,2025-06-13 10:04:51,deposit,1691.09
4,2025-07-29 23:47:37 | user: user1014 | txn: de...,user1014,2025-07-29 23:47:37,deposit,3539.5


In [122]:
data.isnull().sum()

raw_log      0
user_id      0
timestamp    0
action       0
amount       0
dtype: int64

# Extract Currency

In [123]:
currency_pattern = re.compile(r"""
        (?:\|\s*amt[:=]?\d+)? # optional prefix
        ([$€£])                 # currency symbol
        (?:\d+\s*)?                 #optional suffix
""", re.VERBOSE)

text = textwrap.dedent("""
                '2025-07-05 19:18:10::user1069::withdrawal::2995.12::London::iPhone 13',
                'usr:user1076|cashout|€4821.85|Glasgow|2025-07-15 12:56:05|Pixel 6',
                '2025-07-20 05:38:14 >> [user1034] did top-up - amt=€2191.06 - None // dev:iPhone 13',
                '2025-06-13 10:04:51 >> [user1068] did deposit - amt=€1691.09 - Glasgow // dev:None',
                '2025-07-29 23:47:37 | user: user1014 | txn: deposit of £3539.5 from Glasgow | device: iPhone 13',
                '2025-06-23 14:45:58 - user=user1075 - action=debit $1215.74 - ATM: Leeds - device=Samsung Galaxy S10',
                '2025-07-31 06:50:50 | user: user1071 | txn: cashout of $1772.13 from None | device: Nokia 3310',
                '2025-07-07 20:42:12 - user=user1098 - action=deposit €304.0 - ATM: Birmingham - device=Nokia 3310',
                '24/07/2025 22:47:06 ::: user1080 *** PURCHASE ::: amt:951.85$ @ Liverpool <Xiaomi Mi 11>',
                '2025-06-10 20:06:30 >> [user1025] did withdrawal - amt=€3261.07 - London // dev:iPhone 13',
                '14/06/2025 07:25:39 ::: user1019 *** CASHOUT ::: amt:2428.72£ @ Leeds <iPhone 13>',
                'user1093 2025-07-05 14:11:06 withdrawal 4926.56 None Huawei P30',
                '2025-07-05 06:12:48 - user=user1043 - action=withdrawal £4803.93 - ATM: Liverpool - device=iPhone 13',
                '2025-07-23 15:57:12 | user: user1098 | txn: purchase of €2019.47 from Glasgow | device: None',
                '2025-06-23 10:05:51 >> [user1048] did deposit - amt=€2096.71 - Birmingham // dev:Huawei P30',
                'usr:user1034|top-up|$1194.19|Liverpool|2025-06-24 05:06:27|Pixel 6',
                '2025-06-25 07:18:59 - user=user1008 - action=top-up £4645.16 - ATM: London - device=Xiaomi Mi 11',
                '2025-07-17 23:05:52 >> [user1011] did purchase - amt=£2529.92 - Birmingham // dev:iPhone 13',
                '2025-06-22 21:50:40 - user=user1003 - action=refund £4658.85 - ATM: Liverpool - device=Huawei P30',
                '2025-07-04 02:49:11::user1083::transfer::1362.47::Glasgow::Pixel 6',
                'user1077 2025-06-26 03:04:52 refund 2527.54 London iPhone 13',
                '04/07/2025 01:08:12 ::: user1046 *** CASHOUT ::: amt:2804.97£ @ Manchester <Pixel 6>',
                '09/06/2025 22:58:25 ::: user1074 *** WITHDRAWAL ::: amt:4173.14€ @ Liverpool <Xiaomi Mi 11>',
                '2025-07-08 14:07:36 >> [user1086] did refund - amt=£1823.89 - Birmingham // dev:Samsung Galaxy S10',
""")

matches = currency_pattern.findall(text)
print("Matches found:", len(matches), matches)

Matches found: 20 ['€', '€', '€', '£', '$', '$', '€', '$', '€', '£', '£', '€', '€', '$', '£', '£', '£', '£', '€', '£']


In [124]:
data['raw_log'].apply(lambda x: (currency_pattern.findall(x))[0] if (currency_pattern.findall(x)) != [] else np.nan)

0       NaN
1         €
2         €
3         €
4         £
       ... 
7769      £
7770      €
7771    NaN
7772      £
7773    NaN
Name: raw_log, Length: 7774, dtype: object

In [125]:
data['currency'] = data['raw_log'].apply(lambda x: (currency_pattern.findall(x))[0] if (currency_pattern.findall(x)) != [] else np.nan)

In [126]:
data.isnull().sum()

raw_log         0
user_id         0
timestamp       0
action          0
amount          0
currency     2241
dtype: int64

In [127]:
data[data['currency'].isnull()]['raw_log'].head(10).to_list()

['2025-07-05 19:18:10::user1069::withdrawal::2995.12::London::iPhone 13',
 'user1093 2025-07-05 14:11:06 withdrawal 4926.56 None Huawei P30',
 '2025-07-04 02:49:11::user1083::transfer::1362.47::Glasgow::Pixel 6',
 'user1077 2025-06-26 03:04:52 refund 2527.54 London iPhone 13',
 'user1069 2025-07-27 06:27:32 cashout 4497.62 Cardiff iPhone 13',
 '2025-07-05 11:01:54::user1081::purchase::1821.74::Birmingham::None',
 'user1068 2025-06-19 05:02:27 deposit 3599.05 Cardiff Pixel 6',
 'user1020 2025-07-30 02:28:05 cashout 3486.34 Liverpool Samsung Galaxy S10',
 'user1020 2025-07-28 17:58:48 withdrawal 232.02 Manchester Xiaomi Mi 11',
 '2025-06-10 09:06:24::user1005::debit::388.34::Glasgow::Huawei P30']

# Extract Location

In [128]:
location_pattern = re.compile(r"""
        (?:\:\|\s*)?           # optional prefix
        (?P<location>london|glasgow|leeds|birmingham|liverpool|manchester|cardiff)                # location
        (?:\:\s*)?                 #optional suffix
""", re.VERBOSE)

text = textwrap.dedent("""
                '2025-07-05 19:18:10::user1069::withdrawal::2995.12::London::iPhone 13',
                'usr:user1076|cashout|€4821.85|Glasgow|2025-07-15 12:56:05|Pixel 6',
                '2025-07-20 05:38:14 >> [user1034] did top-up - amt=€2191.06 - None // dev:iPhone 13',
                '2025-06-13 10:04:51 >> [user1068] did deposit - amt=€1691.09 - Glasgow // dev:None',
                '2025-07-29 23:47:37 | user: user1014 | txn: deposit of £3539.5 from Glasgow | device: iPhone 13',
                '2025-06-23 14:45:58 - user=user1075 - action=debit $1215.74 - ATM: Leeds - device=Samsung Galaxy S10',
                '2025-07-31 06:50:50 | user: user1071 | txn: cashout of $1772.13 from None | device: Nokia 3310',
                '2025-07-07 20:42:12 - user=user1098 - action=deposit €304.0 - ATM: Birmingham - device=Nokia 3310',
                '24/07/2025 22:47:06 ::: user1080 *** PURCHASE ::: amt:951.85$ @ Liverpool <Xiaomi Mi 11>',
                '2025-06-10 20:06:30 >> [user1025] did withdrawal - amt=€3261.07 - London // dev:iPhone 13',
                '14/06/2025 07:25:39 ::: user1019 *** CASHOUT ::: amt:2428.72£ @ Leeds <iPhone 13>',
                'user1093 2025-07-05 14:11:06 withdrawal 4926.56 None Huawei P30',
                '2025-07-05 06:12:48 - user=user1043 - action=withdrawal £4803.93 - ATM: Liverpool - device=iPhone 13',
                '2025-07-23 15:57:12 | user: user1098 | txn: purchase of €2019.47 from Glasgow | device: None',
                '2025-06-23 10:05:51 >> [user1048] did deposit - amt=€2096.71 - Birmingham // dev:Huawei P30',
                'usr:user1034|top-up|$1194.19|Liverpool|2025-06-24 05:06:27|Pixel 6',
                '2025-06-25 07:18:59 - user=user1008 - action=top-up £4645.16 - ATM: London - device=Xiaomi Mi 11',
                '2025-07-17 23:05:52 >> [user1011] did purchase - amt=£2529.92 - Birmingham // dev:iPhone 13',
                '2025-06-22 21:50:40 - user=user1003 - action=refund £4658.85 - ATM: Liverpool - device=Huawei P30',
                '2025-07-04 02:49:11::user1083::transfer::1362.47::Glasgow::Pixel 6',
                'user1077 2025-06-26 03:04:52 refund 2527.54 London iPhone 13',
                '04/07/2025 01:08:12 ::: user1046 *** CASHOUT ::: amt:2804.97£ @ Manchester <Pixel 6>',
                '09/06/2025 22:58:25 ::: user1074 *** WITHDRAWAL ::: amt:4173.14€ @ Liverpool <Xiaomi Mi 11>',
                '2025-07-08 14:07:36 >> [user1086] did refund - amt=£1823.89 - Birmingham // dev:Samsung Galaxy S10',
""")

matches = location_pattern.findall(text.lower())
print("Matches found:", len(matches), matches)

Matches found: 21 ['london', 'glasgow', 'glasgow', 'glasgow', 'leeds', 'birmingham', 'liverpool', 'london', 'leeds', 'liverpool', 'glasgow', 'birmingham', 'liverpool', 'london', 'birmingham', 'liverpool', 'glasgow', 'london', 'manchester', 'liverpool', 'birmingham']


In [129]:
data['raw_log'].apply(lambda x: (location_pattern.findall(x.lower()))[0] if (location_pattern.findall(x.lower())) != [] else np.nan)

0           london
1          glasgow
2              NaN
3          glasgow
4          glasgow
           ...    
7769         leeds
7770    birmingham
7771       cardiff
7772        london
7773         leeds
Name: raw_log, Length: 7774, dtype: object

In [130]:
data['location'] = data['raw_log'].apply(lambda x: (location_pattern.findall(x.lower()))[0] if (location_pattern.findall(x.lower())) != [] else np.nan)

In [131]:
data.isnull().sum()

raw_log         0
user_id         0
timestamp       0
action          0
amount          0
currency     2241
location      968
dtype: int64

In [132]:
# check raw log if regex missed locations that are present
data[data['location'].isnull()]['raw_log'].to_list()

['2025-07-20 05:38:14 >> [user1034] did top-up - amt=€2191.06 - None // dev:iPhone 13',
 '2025-07-31 06:50:50 | user: user1071 | txn: cashout of $1772.13 from None | device: Nokia 3310',
 'user1093 2025-07-05 14:11:06 withdrawal 4926.56 None Huawei P30',
 '2025-07-24 02:04:16 >> [user1058] did deposit - amt=£1913.33 - None // dev:Samsung Galaxy S10',
 '2025-06-22 11:35:36 >> [user1067] did purchase - amt=$3592.01 - None // dev:Xiaomi Mi 11',
 '2025-07-17 19:56:26 >> [user1040] did transfer - amt=$3718.53 - None // dev:Huawei P30',
 '2025-06-14 11:55:14 >> [user1070] did refund - amt=€3829.3 - None // dev:Pixel 6',
 'usr:user1040|deposit|£2820.28|None|2025-07-28 01:03:02|Pixel 6',
 '25/07/2025 20:53:01 ::: user1050 *** DEPOSIT ::: amt:2168.18$ @ None <Pixel 6>',
 'user1077 2025-07-20 01:34:56 refund 4408.27 None None',
 '2025-06-19 19:01:52 | user: user1025 | txn: purchase of €3453.36 from None | device: Xiaomi Mi 11',
 '2025-06-13 01:12:45 | user: user1011 | txn: refund of £3487.39 fro

# Device

In [133]:
device_pattern = re.compile(r"""
    (?:device\s*[:=]\s*|dev\s*[:=]\s*|\s*<\s*|\s*\:\:\s*|\s*\|\s*|\s+\b)
    (?P<device>(?!None\b)(?:iPhone|Samsung|Pixel|Nokia|Xiaomi|Huawei)(?:\s+(?:\d+|[A-Z]\w*))*)\b                   
""", re.VERBOSE | re.IGNORECASE)

text = textwrap.dedent("""
                '2025-07-05 19:18:10::user1069::withdrawal::2995.12::London::iPhone 13',
                'usr:user1076|cashout|€4821.85|Glasgow|2025-07-15 12:56:05|Pixel 6',
                '2025-07-20 05:38:14 >> [user1034] did top-up - amt=€2191.06 - None // dev:iPhone 13',
                '2025-06-13 10:04:51 >> [user1068] did deposit - amt=€1691.09 - Glasgow // dev:None',
                '2025-07-29 23:47:37 | user: user1014 | txn: deposit of £3539.5 from Glasgow | device: iPhone 13',
                '2025-06-23 14:45:58 - user=user1075 - action=debit $1215.74 - ATM: Leeds - device=Samsung Galaxy S10',
                '2025-07-31 06:50:50 | user: user1071 | txn: cashout of $1772.13 from None | device: Nokia 3310',
                '2025-07-07 20:42:12 - user=user1098 - action=deposit €304.0 - ATM: Birmingham - device=Nokia 3310',
                '24/07/2025 22:47:06 ::: user1080 *** PURCHASE ::: amt:951.85$ @ Liverpool <Xiaomi Mi 11>',
                '2025-06-10 20:06:30 >> [user1025] did withdrawal - amt=€3261.07 - London // dev:iPhone 13',
                '14/06/2025 07:25:39 ::: user1019 *** CASHOUT ::: amt:2428.72£ @ Leeds <iPhone 13>',
                'user1093 2025-07-05 14:11:06 withdrawal 4926.56 None Huawei P30',
                '2025-07-05 06:12:48 - user=user1043 - action=withdrawal £4803.93 - ATM: Liverpool - device=iPhone 13',
                '2025-07-23 15:57:12 | user: user1098 | txn: purchase of €2019.47 from Glasgow | device: None',
                '2025-06-23 10:05:51 >> [user1048] did deposit - amt=€2096.71 - Birmingham // dev:Huawei P30',
                'usr:user1034|top-up|$1194.19|Liverpool|2025-06-24 05:06:27|Pixel 6',
                '2025-06-25 07:18:59 - user=user1008 - action=top-up £4645.16 - ATM: London - device=Xiaomi Mi 11',
                '2025-07-17 23:05:52 >> [user1011] did purchase - amt=£2529.92 - Birmingham // dev:iPhone 13',
                '2025-06-22 21:50:40 - user=user1003 - action=refund £4658.85 - ATM: Liverpool - device=Huawei P30',
                '2025-07-04 02:49:11::user1083::transfer::1362.47::Glasgow::Pixel 6',
                'user1077 2025-06-26 03:04:52 refund 2527.54 London iPhone 13',
                '04/07/2025 01:08:12 ::: user1046 *** CASHOUT ::: amt:2804.97£ @ Manchester <Pixel 6>',
                '09/06/2025 22:58:25 ::: user1074 *** WITHDRAWAL ::: amt:4173.14€ @ Liverpool <Xiaomi Mi 11>',
                '2025-07-08 14:07:36 >> [user1086] did refund - amt=£1823.89 - Birmingham // dev:Samsung Galaxy S10',
""")

matches = device_pattern.findall(text)
print("Matches found:", len(matches), matches)

Matches found: 22 ['iPhone 13', 'Pixel 6', 'iPhone 13', 'iPhone 13', 'Samsung Galaxy S10', 'Nokia 3310', 'Nokia 3310', 'Xiaomi Mi 11', 'iPhone 13', 'iPhone 13', 'Huawei P30', 'iPhone 13', 'Huawei P30', 'Pixel 6', 'Xiaomi Mi 11', 'iPhone 13', 'Huawei P30', 'Pixel 6', 'iPhone 13', 'Pixel 6', 'Xiaomi Mi 11', 'Samsung Galaxy S10']


In [134]:
data['raw_log'].apply(lambda x: device_pattern.findall(x)[0] if device_pattern.findall(x) != [] else np.nan)

0          iPhone 13
1            Pixel 6
2          iPhone 13
3                NaN
4          iPhone 13
            ...     
7769         Pixel 6
7770      Nokia 3310
7771      Nokia 3310
7772    Xiaomi Mi 11
7773    Xiaomi Mi 11
Name: raw_log, Length: 7774, dtype: object

In [135]:
data['device'] = data['raw_log'].apply(lambda x: device_pattern.findall(x)[0] if device_pattern.findall(x) != [] else np.nan)

In [136]:
data.isnull().sum()

raw_log         0
user_id         0
timestamp       0
action          0
amount          0
currency     2241
location      968
device       1081
dtype: int64

In [137]:
# check the raw logs of those with null device
data[data['device'].isnull()]['raw_log'].to_list()

['2025-06-13 10:04:51 >> [user1068] did deposit - amt=€1691.09 - Glasgow // dev:None',
 '2025-07-23 15:57:12 | user: user1098 | txn: purchase of €2019.47 from Glasgow | device: None',
 '30/07/2025 04:54:02 ::: user1096 *** DEPOSIT ::: amt:1769.04€ @ Leeds <None>',
 '2025-07-05 11:01:54::user1081::purchase::1821.74::Birmingham::None',
 'user1077 2025-07-20 01:34:56 refund 4408.27 None None',
 '2025-07-21 12:27:25 | user: user1087 | txn: refund of €1702.35 from Cardiff | device: None',
 'usr:user1087|withdrawal|€4447.23|Birmingham|2025-07-10 09:46:29|None',
 'user1088 2025-07-20 22:06:28 transfer 375.07 Birmingham None',
 'user1010 2025-06-14 17:47:12 top-up 1874.83 Glasgow None',
 '2025-06-26 08:30:06 >> [user1065] did transfer - amt=£850.75 - Leeds // dev:None',
 '2025-07-16 11:09:32::user1059::withdrawal::1545.36::London::None',
 'user1028 2025-07-11 05:36:37 debit 1843.56 Cardiff None',
 'usr:user1020|withdrawal|£1387.1|Liverpool|2025-07-07 22:10:59|None',
 'usr:user1011|purchase|€40

In [138]:
# since those with null device are "None" on the raw log, replace nan with None
data['device'] = data['device'].fillna('None')

In [139]:
data.isna().sum()

raw_log         0
user_id         0
timestamp       0
action          0
amount          0
currency     2241
location      968
device          0
dtype: int64

# Extract ATM flag

In [140]:
atm_pattern = re.compile(r"""
        atm            
""", re.VERBOSE | re.IGNORECASE)
text = textwrap.dedent("""
                '2025-07-05 19:18:10::user1069::withdrawal::2995.12::London::iPhone 13',
                'usr:user1076|cashout|€4821.85|Glasgow|2025-07-15 12:56:05|Pixel 6',
                '2025-07-20 05:38:14 >> [user1034] did top-up - amt=€2191.06 - None // dev:iPhone 13',
                '2025-06-13 10:04:51 >> [user1068] did deposit - amt=€1691.09 - Glasgow // dev:None',
                '2025-07-29 23:47:37 | user: user1014 | txn: deposit of £3539.5 from Glasgow | device: iPhone 13',
                '2025-06-23 14:45:58 - user=user1075 - action=debit $1215.74 - ATM: Leeds - device=Samsung Galaxy S10',
                '2025-07-31 06:50:50 | user: user1071 | txn: cashout of $1772.13 from None | device: Nokia 3310',
                '2025-07-07 20:42:12 - user=user1098 - action=deposit €304.0 - ATM: Birmingham - device=Nokia 3310',
                '24/07/2025 22:47:06 ::: user1080 *** PURCHASE ::: amt:951.85$ @ Liverpool <Xiaomi Mi 11>',
                '2025-06-10 20:06:30 >> [user1025] did withdrawal - amt=€3261.07 - London // dev:iPhone 13',
                '14/06/2025 07:25:39 ::: user1019 *** CASHOUT ::: amt:2428.72£ @ Leeds <iPhone 13>',
                'user1093 2025-07-05 14:11:06 withdrawal 4926.56 None Huawei P30',
                '2025-07-05 06:12:48 - user=user1043 - action=withdrawal £4803.93 - ATM: Liverpool - device=iPhone 13',
                '2025-07-23 15:57:12 | user: user1098 | txn: purchase of €2019.47 from Glasgow | device: None',
                '2025-06-23 10:05:51 >> [user1048] did deposit - amt=€2096.71 - Birmingham // dev:Huawei P30',
                'usr:user1034|top-up|$1194.19|Liverpool|2025-06-24 05:06:27|Pixel 6',
                '2025-06-25 07:18:59 - user=user1008 - action=top-up £4645.16 - ATM: London - device=Xiaomi Mi 11',
                '2025-07-17 23:05:52 >> [user1011] did purchase - amt=£2529.92 - Birmingham // dev:iPhone 13',
                '2025-06-22 21:50:40 - user=user1003 - action=refund £4658.85 - ATM: Liverpool - device=Huawei P30',
                '2025-07-04 02:49:11::user1083::transfer::1362.47::Glasgow::Pixel 6',
                'user1077 2025-06-26 03:04:52 refund 2527.54 London iPhone 13',
                '04/07/2025 01:08:12 ::: user1046 *** CASHOUT ::: amt:2804.97£ @ Manchester <Pixel 6>',
                '09/06/2025 22:58:25 ::: user1074 *** WITHDRAWAL ::: amt:4173.14€ @ Liverpool <Xiaomi Mi 11>',
                '2025-07-08 14:07:36 >> [user1086] did refund - amt=£1823.89 - Birmingham // dev:Samsung Galaxy S10',
""")

matches = atm_pattern.findall(text.lower())
print("Matches found:", len(matches), matches)

Matches found: 5 ['atm', 'atm', 'atm', 'atm', 'atm']


In [141]:
data['atm'] = data['raw_log'].apply(lambda x: atm_pattern.findall(x.lower())[0] if atm_pattern.findall(x.lower()) != [] else np.nan)

In [142]:
data

,raw_log,user_id,timestamp,action,amount,currency,location,device,atm
0,2025-07-05 19:18:10::user1069::withdrawal::299...,user1069,2025-07-05 19:18:10,withdrawal,2995.12,NaN,london,iPhone 13,NaN
1,usr:user1076|cashout|€4821.85|Glasgow|2025-07-...,user1076,2025-07-15 12:56:05,cashout,4821.85,€,glasgow,Pixel 6,NaN
2,2025-07-20 05:38:14 >> [user1034] did top-up -...,user1034,2025-07-20 05:38:14,top-up,2191.06,€,NaN,iPhone 13,NaN
3,2025-06-13 10:04:51 >> [user1068] did deposit ...,user1068,2025-06-13 10:04:51,deposit,1691.09,€,glasgow,None,NaN
4,2025-07-29 23:47:37 | user: user1014 | txn: de...,user1014,2025-07-29 23:47:37,deposit,3539.5,£,glasgow,iPhone 13,NaN
...,...,...,...,...,...,...,...,...,...
7769,usr:user1058|debit|£3992.06|Leeds|2025-06-10 0...,user1058,2025-06-10 03:55:41,debit,3992.06,£,leeds,Pixel 6,NaN
7770,08/07/2025 08:50:09 ::: user1069 *** WITHDRAWA...,user1069,08/07/2025 08:50:09,withdrawal,1401.58,€,birmingham,Nokia 3310,NaN
7771,2025-07-31 07:41:34::user1024::deposit::1331.2...,user1024,2025-07-31 07:41:34,deposit,1331.2,NaN,cardiff,Nokia 3310,NaN
7772,2025-06-23 05:14:03 >> [user1029] did withdraw...,user1029,2025-06-23 05:14:03,withdrawal,4203.63,£,london,Xiaomi Mi 11,NaN


## Check for pattern in missing data

### Missing Currency

In [143]:
# Check if a currency is tied to a particular location

for location in data['location'].unique():
    print(location)
    print(data[data['location']== location]['currency'].value_counts())
    print(" ")

london
currency
€    241
$    226
£    201
Name: count, dtype: int64
 
glasgow
currency
$    251
£    239
€    219
Name: count, dtype: int64
 
nan
Series([], Name: count, dtype: int64)
 
leeds
currency
$    237
€    235
£    228
Name: count, dtype: int64
 
birmingham
currency
£    231
$    224
€    214
Name: count, dtype: int64
 
liverpool
currency
€    262
£    236
$    192
Name: count, dtype: int64
 
manchester
currency
€    248
$    232
£    228
Name: count, dtype: int64
 
cardiff
currency
£    236
$    234
€    219
Name: count, dtype: int64
 


In [144]:
# Check if a currency is tied to a particular action

for action in data['action'].unique():
    print(action)
    print(data[data['action']== action]['currency'].value_counts())
    print(" ")

withdrawal
currency
€    237
£    228
$    228
Name: count, dtype: int64
 
cashout
currency
€    268
£    225
$    222
Name: count, dtype: int64
 
top-up
currency
$    238
£    237
€    231
Name: count, dtype: int64
 
deposit
currency
$    258
€    237
£    225
Name: count, dtype: int64
 
debit
currency
$    246
£    241
€    228
Name: count, dtype: int64
 
purchase
currency
€    240
£    228
$    211
Name: count, dtype: int64
 
refund
currency
£    223
$    216
€    198
Name: count, dtype: int64
 
transfer
currency
£    227
€    224
$    217
Name: count, dtype: int64
 


In [145]:
# check if missing currency can be associated strongly with missing values in any other features
data[data['currency'].isnull()].isnull().sum()

raw_log         0
user_id         0
timestamp       0
action          0
amount          0
currency     2241
location      268
device          0
atm          2241
dtype: int64

**OBSERVATION**
- The Currencies are missing at random

**RECOMMENDATION**
- Dropping missing records are unadvisible as they could potentially be a significant portion of anomalies 
- Fill with "None" to reperesent the absence of currency in the transaction record

In [146]:
data['currency'] = data['currency'].fillna('None')

In [147]:
data.isnull().sum()

raw_log         0
user_id         0
timestamp       0
action          0
amount          0
currency        0
location      968
device          0
atm          6685
dtype: int64

### Observe missing values in Location

In [148]:
data[data['location'].isnull()]

,raw_log,user_id,timestamp,action,amount,currency,location,device,atm
2,2025-07-20 05:38:14 >> [user1034] did top-up -...,user1034,2025-07-20 05:38:14,top-up,2191.06,€,NaN,iPhone 13,NaN
6,2025-07-31 06:50:50 | user: user1071 | txn: ca...,user1071,2025-07-31 06:50:50,cashout,1772.13,$,NaN,Nokia 3310,NaN
11,user1093 2025-07-05 14:11:06 withdrawal 4926.5...,user1093,2025-07-05 14:11:06,withdrawal,4926.56,None,NaN,Huawei P30,NaN
27,2025-07-24 02:04:16 >> [user1058] did deposit ...,user1058,2025-07-24 02:04:16,deposit,1913.33,£,NaN,Samsung Galaxy S10,NaN
37,2025-06-22 11:35:36 >> [user1067] did purchase...,user1067,2025-06-22 11:35:36,purchase,3592.01,$,NaN,Xiaomi Mi 11,NaN
...,...,...,...,...,...,...,...,...,...
7724,usr:user1082|debit|€4920.89|None|2025-06-29 22...,user1082,2025-06-29 22:28:32,debit,4920.89,€,NaN,Samsung Galaxy S10,NaN
7733,2025-07-10 16:54:57 | user: user1074 | txn: de...,user1074,2025-07-10 16:54:57,deposit,4511.11,€,NaN,Xiaomi Mi 11,NaN
7742,2025-06-15 14:34:49 | user: user1065 | txn: de...,user1065,2025-06-15 14:34:49,debit,14.52,€,NaN,Xiaomi Mi 11,NaN
7760,usr:user1011|cashout|€3725.69|None|2025-06-27 ...,user1011,2025-06-27 23:20:52,cashout,3725.69,€,NaN,Samsung Galaxy S10,NaN


In [149]:
# Check if a particular user has the most missing location: Could potentially raise suspicision 
data[data['location'].isnull()]['user_id'].value_counts()

user_id
user1020    30
user1081    27
user1048    27
user1058    25
user1033    25
            ..
user1022     2
user1085     2
user1079     2
user1024     2
user1073     1
Name: count, Length: 86, dtype: int64

In [150]:
# check the number of transaction made by user1020 that has the most missing location
data[data['user_id'] == 'user1020'].shape

(238, 9)

**OBSERVATION/Recommendation**
-  Replace missing value in location with "None" to indicate the lack of such. 
- This would allow the ML algorithm detect if that pattern is an anomaly 

In [151]:
data['location'] = data['location'].fillna('None')

In [152]:
data.isnull().sum()

raw_log         0
user_id         0
timestamp       0
action          0
amount          0
currency        0
location        0
device          0
atm          6685
dtype: int64

### Observe missing atm flag

In [153]:
data.head()

,raw_log,user_id,timestamp,action,amount,currency,location,device,atm
0,2025-07-05 19:18:10::user1069::withdrawal::299...,user1069,2025-07-05 19:18:10,withdrawal,2995.12,None,london,iPhone 13,NaN
1,usr:user1076|cashout|€4821.85|Glasgow|2025-07-...,user1076,2025-07-15 12:56:05,cashout,4821.85,€,glasgow,Pixel 6,NaN
2,2025-07-20 05:38:14 >> [user1034] did top-up -...,user1034,2025-07-20 05:38:14,top-up,2191.06,€,None,iPhone 13,NaN
3,2025-06-13 10:04:51 >> [user1068] did deposit ...,user1068,2025-06-13 10:04:51,deposit,1691.09,€,glasgow,None,NaN
4,2025-07-29 23:47:37 | user: user1014 | txn: de...,user1014,2025-07-29 23:47:37,deposit,3539.5,£,glasgow,iPhone 13,NaN


In [154]:
data[data['atm'].isnull()]['raw_log'].to_list()

['2025-07-05 19:18:10::user1069::withdrawal::2995.12::London::iPhone 13',
 'usr:user1076|cashout|€4821.85|Glasgow|2025-07-15 12:56:05|Pixel 6',
 '2025-07-20 05:38:14 >> [user1034] did top-up - amt=€2191.06 - None // dev:iPhone 13',
 '2025-06-13 10:04:51 >> [user1068] did deposit - amt=€1691.09 - Glasgow // dev:None',
 '2025-07-29 23:47:37 | user: user1014 | txn: deposit of £3539.5 from Glasgow | device: iPhone 13',
 '2025-07-31 06:50:50 | user: user1071 | txn: cashout of $1772.13 from None | device: Nokia 3310',
 '24/07/2025 22:47:06 ::: user1080 *** PURCHASE ::: amt:951.85$ @ Liverpool <Xiaomi Mi 11>',
 '2025-06-10 20:06:30 >> [user1025] did withdrawal - amt=€3261.07 - London // dev:iPhone 13',
 '14/06/2025 07:25:39 ::: user1019 *** CASHOUT ::: amt:2428.72£ @ Leeds <iPhone 13>',
 'user1093 2025-07-05 14:11:06 withdrawal 4926.56 None Huawei P30',
 '2025-07-23 15:57:12 | user: user1098 | txn: purchase of €2019.47 from Glasgow | device: None',
 '2025-06-23 10:05:51 >> [user1048] did depo

# Clean and Explore Data Further

In [155]:
data.describe().T

,count,unique,top,freq
raw_log,7774,7774,2025-07-05 19:18:10::user1069::withdrawal::299...,1
user_id,7774,86,user1020,238
timestamp,7774,7771,2025-07-01 04:57:41,2
action,7774,8,cashout,1015
amount,7774,7709,2484.39,2
currency,7774,4,None,2241
location,7774,8,leeds,995
device,7774,7,Xiaomi Mi 11,1140
atm,1089,1,atm,1089


In [156]:
data.dtypes

raw_log      object
user_id      object
timestamp    object
action       object
amount       object
currency     object
location     object
device       object
atm          object
dtype: object

In [157]:
# Correct the data type of amount
data['amount'] = data['amount'].astype(float)

In [158]:
# convert data type to datetime
data['timestamp'] = pd.to_datetime(data['timestamp'], format='mixed', dayfirst=True  )

In [159]:
data.dtypes

raw_log              object
user_id              object
timestamp    datetime64[ns]
action               object
amount              float64
currency             object
location             object
device               object
atm                  object
dtype: object

In [160]:
# Convert all features to lower case
for col in data.select_dtypes(include='object').columns.to_list():
    data[col] = data[col].str.lower()

In [161]:
data.dtypes

raw_log              object
user_id              object
timestamp    datetime64[ns]
action               object
amount              float64
currency             object
location             object
device               object
atm                  object
dtype: object

In [162]:
data.head()

,raw_log,user_id,timestamp,action,amount,currency,location,device,atm
0,2025-07-05 19:18:10::user1069::withdrawal::299...,user1069,2025-07-05 19:18:10,withdrawal,2995.12,none,london,iphone 13,NaN
1,usr:user1076|cashout|€4821.85|glasgow|2025-07-...,user1076,2025-07-15 12:56:05,cashout,4821.85,€,glasgow,pixel 6,NaN
2,2025-07-20 05:38:14 >> [user1034] did top-up -...,user1034,2025-07-20 05:38:14,top-up,2191.06,€,none,iphone 13,NaN
3,2025-06-13 10:04:51 >> [user1068] did deposit ...,user1068,2025-06-13 10:04:51,deposit,1691.09,€,glasgow,none,NaN
4,2025-07-29 23:47:37 | user: user1014 | txn: de...,user1014,2025-07-29 23:47:37,deposit,3539.50,£,glasgow,iphone 13,NaN


In [163]:
data = data.sort_values('timestamp').reset_index(drop=True)

In [164]:
data

,raw_log,user_id,timestamp,action,amount,currency,location,device,atm
0,usr:user1093|withdrawal|€4259.25|cardiff|2025-...,user1093,2025-06-01 11:41:41,withdrawal,4259.25,€,cardiff,none,NaN
1,2025-06-01 12:03:31 - user=user1000 - action=c...,user1000,2025-06-01 12:03:31,cashout,2981.21,$,london,samsung galaxy s10,atm
2,usr:user1019|purchase|£4461.41|cardiff|2025-06...,user1019,2025-06-01 12:14:07,purchase,4461.41,£,cardiff,huawei p30,NaN
3,2025-06-01 12:44:19::user1048::deposit::2575.6...,user1048,2025-06-01 12:44:19,deposit,2575.66,none,leeds,iphone 13,NaN
4,01/06/2025 12:48:18 ::: user1014 *** refund ::...,user1014,2025-06-01 12:48:18,refund,3949.98,$,none,huawei p30,NaN
...,...,...,...,...,...,...,...,...,...
7769,user1000 2025-07-31 09:41:19 withdrawal 966.27...,user1000,2025-07-31 09:41:19,withdrawal,966.27,none,glasgow,xiaomi mi 11,NaN
7770,usr:user1077|deposit|€730.81|leeds|2025-07-31 ...,user1077,2025-07-31 09:48:20,deposit,730.81,€,leeds,none,NaN
7771,2025-07-31 09:50:41 | user: user1043 | txn: to...,user1043,2025-07-31 09:50:41,top-up,4718.76,$,liverpool,nokia 3310,NaN
7772,2025-07-31 10:20:49 | user: user1087 | txn: tr...,user1087,2025-07-31 10:20:49,transfer,3756.03,€,manchester,iphone 13,NaN


In [165]:
data['timestamp'].min(), data['timestamp'].max()

(Timestamp('2025-06-01 11:41:41'), Timestamp('2025-07-31 11:14:57'))

In [166]:
data.dtypes

raw_log              object
user_id              object
timestamp    datetime64[ns]
action               object
amount              float64
currency             object
location             object
device               object
atm                  object
dtype: object

In [167]:
# save final clean data
data.to_csv('../fraud_tools_team_ds_test/clean_data.csv', index=False)